In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
from time import sleep, time
from requests import get
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})
import plotly.graph_objects as go
from dash import Dash
from dash_bootstrap_templates import load_figure_template
from jupyter_dash import JupyterDash
from IPython import get_ipython
import dash_bootstrap_components as dbc

load_figure_template("darkly")
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"
try:
    shell = str(get_ipython())
    if "ZMQInteractiveShell" in shell:
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    elif shell == "TerminalInteractiveShell":
        app = JupyterDash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
    else:
        app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])
except NameError:
    app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY, dbc_css])

bg_color = "#0b0b18"

%load_ext autoreload
%autoreload 2

In [25]:
BINANCE_TIMEFRAMES = ["1m", "3m", "5m"]
TIMEFRAMES_IN_MINUTES = [1, 3, 5]


def get_timeframe_in_ms(timeframe: str):
    return int(timedelta(minutes=TIMEFRAMES_IN_MINUTES[BINANCE_TIMEFRAMES.index(timeframe)]).seconds * 1000)


def get_busdm_candles(
    symbol: str,
    timeframe: str,
    since_date_ms: int = None,
    until_date_ms: int = None,
    candles_to_dl: int = 1500,
    limit: int = 1500,
):
    timeframe_in_ms = get_timeframe_in_ms(timeframe=timeframe)
    candles_to_dl_ms = candles_to_dl * timeframe_in_ms

    if until_date_ms is None:
        if since_date_ms is None:
            until_date_ms = int(time() * 1000) - timeframe_in_ms
            since_date_ms = until_date_ms - candles_to_dl_ms
        else:
            until_date_ms = since_date_ms + candles_to_dl_ms - 5000
    else:
        if since_date_ms is None:
            since_date_ms = until_date_ms - candles_to_dl_ms
            until_date_ms -= 5000

    b_candles = []
    params = {
        "symbol": symbol,
        "interval": timeframe,
        "startTime": since_date_ms,
        "endTime": until_date_ms,
        "limit": limit,
    }

    while params["startTime"] + timeframe_in_ms < until_date_ms:
        try:
            b_data = get(url="https://fapi.binance.com/fapi/v1/klines", params=params).json()
            last_candle_time_ms = b_data[-1][0]
            if last_candle_time_ms == params["startTime"]:
                sleep(0.3)
            else:
                b_candles.extend(b_data)
                params["startTime"] = last_candle_time_ms + 2000
        except Exception as e:
            raise Exception(f"get_busdm_candles -> {e}")
    candles_np = np.array(b_candles, dtype=np.float_)[:, :5]
    return candles_np

In [10]:
def busdm_to_df(candles: np.array):
    candles_df = pd.DataFrame(candles, columns=['timestamp', 'open', 'high', 'low', 'close'])
    candles_df['timestamp'] = candles_df['timestamp'].astype(dtype=np.int64)
    candles_df.set_index(pd.to_datetime(candles_df['timestamp'], unit='ms'), inplace=True)
    candles_df.index.rename('datetime', inplace=True)
    return candles_df

In [23]:
def plot_busdm_candles(candles: np.array):
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=pd.to_datetime(candles[:, 0], unit="ms"),
                open=candles[:, 1],
                high=candles[:, 2],
                low=candles[:, 3],
                close=candles[:, 4],
            )
        ]
    )
    fig.update_layout(height=600, xaxis_rangeslider_visible=False)
    fig.show()

In [26]:
candles_busdm = get_busdm_candles(symbol='BTCUSDT', timeframe='5m', candles_to_dl=30)

In [28]:
candles_busdm_df = busdm_to_df(candles=candles_busdm)
candles_busdm_df

,timestamp,open,high,low,close
datetime,,,,,
2023-11-12 13:00:00,1699794000000,37035.9,37036.0,36955.0,37002.2
2023-11-12 13:05:00,1699794300000,37002.2,37018.2,36973.6,37015.9
2023-11-12 13:10:00,1699794600000,37016.0,37079.3,37010.3,37070.0
2023-11-12 13:15:00,1699794900000,37070.0,37166.1,37070.0,37139.1
2023-11-12 13:20:00,1699795200000,37139.3,37157.4,37103.9,37108.5
2023-11-12 13:25:00,1699795500000,37108.5,37135.0,37105.2,37121.5
2023-11-12 13:30:00,1699795800000,37121.5,37148.6,37102.0,37116.6
2023-11-12 13:35:00,1699796100000,37116.6,37124.5,37081.5,37083.5
2023-11-12 13:40:00,1699796400000,37083.4,37092.0,37062.1,37078.7


In [24]:
plot_busdm_candles(candles=candles_busdm)